In [1]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD, Adam
from keras.models import Sequential,Input,Model,load_model
from keras.layers import Dense, Dropout, Flatten, GaussianNoise
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import TensorBoard
from keras import regularizers

Using TensorFlow backend.


In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img

def preprocess_input_vgg(x):
    """Wrapper around keras.applications.vgg16.preprocess_input()
    to make it compatible for use with keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument.
    
    Parameters
    ----------
    x : a numpy 3darray (a single image to be preprocessed)
    
    Note we cannot pass keras.applications.vgg16.preprocess_input()
    directly to to keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument because the former expects a
    4D tensor whereas the latter expects a 3D tensor. Hence the
    existence of this wrapper.
    
    Returns a numpy 3darray (the preprocessed image).
    
    """
    from keras.applications.inception_v3 import preprocess_input
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\akiec\train',
                                                    target_size=[140, 140],
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)
validation_generator = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\akiec\test',
                                                              target_size=[140, 140],
                                                              batch_size=32,
                                                              class_mode='categorical')

Found 541 images belonging to 2 classes.
Found 136 images belonging to 2 classes.


In [3]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
inc_model = InceptionV3(weights='imagenet', include_top=False)

# get layers and add average pooling layer
x = inc_model.output
x = GlobalAveragePooling2D()(x)

# add fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
# add output layer
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=inc_model.input, outputs=predictions)

# freeze pre-trained model area's layer
for layer in inc_model.layers:
    layer.trainable = False

# update the weight that are added
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'] )
model.fit_generator(train_generator,
                    steps_per_epoch=64,
                    nb_epoch=1,
                    validation_data=validation_generator,
                    validation_steps=32)

# choose the layers which are updated by training
#layer_num = len(model.layers)
#for layer in model.layers[:248]:
#    layer.trainable = False

for layer in model.layers:
    layer.trainable = True

# training
model.compile(optimizer=SGD(lr=1e-4, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy', 'mae'])

C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:30: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=64, validation_data=<keras_pre..., validation_steps=32, epochs=1)`


Epoch 1/1
64/64 [==============================] - 46s 713ms/step - loss: 1.4793 - acc: 0.5702 - val_loss: 0.6068 - val_acc: 0.6943


In [4]:
history = model.fit_generator(train_generator,
                    steps_per_epoch=64,
                    nb_epoch=10,
                    shuffle = True,
                    validation_data=validation_generator,
                    validation_steps=16)

C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
C:\Users\7\Anaconda3\envs\tg-gpu-ptchd\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=64, shuffle=True, validation_data=<keras_pre..., validation_steps=16, epochs=10)`
  


Epoch 1/10
64/64 [==============================] - 40s 621ms/step - loss: 0.5911 - acc: 0.6993 - mean_absolute_error: 0.4112 - val_loss: 0.6163 - val_acc: 0.6455 - val_mean_absolute_error: 0.4458
Epoch 2/10
64/64 [==============================] - 29s 449ms/step - loss: 0.5650 - acc: 0.7184 - mean_absolute_error: 0.4000 - val_loss: 0.5977 - val_acc: 0.6341 - val_mean_absolute_error: 0.4324
Epoch 3/10
64/64 [==============================] - 29s 446ms/step - loss: 0.5555 - acc: 0.7292 - mean_absolute_error: 0.3902 - val_loss: 0.5785 - val_acc: 0.6886 - val_mean_absolute_error: 0.4152
Epoch 4/10
64/64 [==============================] - 27s 430ms/step - loss: 0.5201 - acc: 0.7630 - mean_absolute_error: 0.3683 - val_loss: 0.5764 - val_acc: 0.6545 - val_mean_absolute_error: 0.4134
Epoch 5/10
64/64 [==============================] - 27s 420ms/step - loss: 0.5085 - acc: 0.7619 - mean_absolute_error: 0.3570 - val_loss: 0.5575 - val_acc: 0.7236 - val_mean_absolute_error: 0.4011
Epoch 6/10
64/6

In [4]:
import pandas as pd

predict_gen = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\akiec\test',
                                                              target_size=[140,140],
                                                              batch_size=136,
                                                              class_mode='categorical')

X_val_sample, res = next(predict_gen)
y_pred = model.predict(X_val_sample)


Found 506 images belonging to 6 classes.


In [5]:
model.evaluate_generator(validation_generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)

IndexError: list index out of range

In [10]:
onedtrue = []
onedpred = []
for true, pred in zip(res, y_pred):
    onedtrue.append(np.argmax(true))
    onedpred.append(np.argmax(pred))

In [11]:
from sklearn.metrics import confusion_matrix
confusion_matrix(onedtrue, onedpred)

array([[ 44,  22,   0,   0,   0,   0],
       [ 16,  87,   0,   0,   0,   0],
       [ 10,  93,   0,   0,   0,   0],
       [  7,  96,   0,   0,   0,   0],
       [  0, 103,   0,   0,   0,   0],
       [  0,  28,   0,   0,   0,   0]], dtype=int64)

In [12]:
from sklearn.metrics import classification_report
target_names = ['akiec', 'akiec-non']
print(classification_report(onedtrue, onedpred, target_names=target_names))

ValueError: Number of classes, 6, does not match size of target_names, 2. Try specifying the labels parameter

In [29]:
model.save('akiec-7867-40epc.h5')

# Ensemble

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD, Adam
from keras.models import Sequential,Input,Model,load_model
from keras.layers import Dense, Dropout, Flatten, GaussianNoise
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import TensorBoard
from keras import regularizers

Using TensorFlow backend.


In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img

def preprocess_input_vgg(x):
    """Wrapper around keras.applications.vgg16.preprocess_input()
    to make it compatible for use with keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument.
    
    Parameters
    ----------
    x : a numpy 3darray (a single image to be preprocessed)
    
    Note we cannot pass keras.applications.vgg16.preprocess_input()
    directly to to keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument because the former expects a
    4D tensor whereas the latter expects a 3D tensor. Hence the
    existence of this wrapper.
    
    Returns a numpy 3darray (the preprocessed image).
    
    """
    from keras.applications.inception_v3 import preprocess_input
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\akiec\train',
                                                    target_size=[140, 140],
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)
validation_generator = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\binary dataset\akiec\test',
                                                              target_size=[140, 140],
                                                              batch_size=32,
                                                              class_mode='categorical')

Found 541 images belonging to 2 classes.
Found 136 images belonging to 2 classes.


In [3]:
model = load_model(r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\ensemble\akiec-7867-40epc.h5')

In [4]:
predictions = []
for k in range(506):
    predictions.append([])

In [5]:
import pandas as pd

predict_gen = validation_datagen.flow_from_directory(directory=r'C:\Users\7\Desktop\Desktop\dataset\cancer mnist\unbalanced\test',
                                                              target_size=[140,140],
                                                              batch_size=506,
                                                              shuffle = False,
                                                              class_mode='categorical')

X_val_sample, res = next(predict_gen)
y_pred = model.predict(X_val_sample)


Found 506 images belonging to 6 classes.


In [6]:
for index,y in enumerate(y_pred):
    predictions[index].append(y[0])

In [10]:
import pickle
with open("pred_akiec.txt", "wb") as fp:   #Pickling
    pickle.dump(predictions, fp)